In [27]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

In [2]:
path = './data/'

data = pd.read_csv(path + 'train.csv')

print(len(data))
print(data.columns)

1647
Index(['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type',
       'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine',
       'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height',
       'Seating Capacity', 'Fuel Tank Capacity'],
      dtype='object')


In [3]:
data.head()

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,BMW,3-Series 320d,800000,2012,75576,Diesel,Automatic,Mumbai,White,Second,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BMW,X1 sDrive20d xLine,2199000,2016,77000,Diesel,Automatic,Surat,Black,First,Individual,1995 cc,184 bhp @ 4000 rpm,350 Nm @ 1750 rpm,RWD,4454.0,2044.0,1545.0,5.0,63.0
2,Mahindra,XUV500 W4 1.99,800000,2017,112000,Diesel,Manual,Muzaffurpur,Silver,First,Individual,1997 cc,138 bhp @ 3750 rpm,320 Nm @ 1600 rpm,FWD,4585.0,1890.0,1785.0,7.0,70.0
3,Mercedes-Benz,GLS 400d 4MATIC,12900000,2021,3000,Diesel,Automatic,Delhi,White,First,Individual,2925 cc,326 bhp @ 3600 rpm,700 Nm @ 1200 rpm,AWD,5207.0,2157.0,1823.0,7.0,90.0
4,Toyota,Fortuner 2.8 4x2 AT [2016-2020],3499000,2019,73000,Diesel,Automatic,Mumbai,White,First,Individual,2755 cc,174 bhp @ 3400 rpm,450 Nm @ 1600 rpm,RWD,4795.0,1855.0,1835.0,7.0,80.0


# PREPROCESS DATA

## 1. Normalization

In [4]:
data.dropna(how='all')

data["Engine"] = data["Engine"].str.replace(' cc', '', regex=False).astype(float)

data[['Max Power BHP', 'Max Power RPM']] = data['Max Power'].str.extract(r'(\d+)\s*bhp\s*@\s*(\d+)\s*rpm')
data['Max Power BHP'] = pd.to_numeric(data['Max Power BHP'], errors='coerce')
data['Max Power RPM'] = pd.to_numeric(data['Max Power RPM'], errors='coerce')

data[['Max Torque Nm', 'Max Torque RPM']] = data['Max Torque'].str.extract(r'(\d+)\s*Nm\s*@\s*(\d+)\s*rpm')
data['Max Torque Nm'] = pd.to_numeric(data['Max Torque Nm'], errors='coerce')
data['Max Torque RPM'] = pd.to_numeric(data['Max Torque RPM'], errors='coerce')

data = data.drop('Max Power', axis=1)
data = data.drop('Max Torque', axis=1)



Số lượng dữ liệu bị thiếu ở các cột

In [5]:
data.isna().sum()


Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                 65
Drivetrain            111
Length                 52
Width                  52
Height                 52
Seating Capacity       52
Fuel Tank Capacity     90
Max Power BHP         172
Max Power RPM         172
Max Torque Nm         168
Max Torque RPM        168
dtype: int64

## 2. Xử lí dữ liệu bị thiếu

In [6]:
def drop_na(df : pd.DataFrame):
    return df.dropna(how='any')

### 2.1 Fill missing values with mean

In [7]:
def fill_mean(df : pd.DataFrame):
    return df.apply(lambda x : x.fillna(x.mean()) if x.dtype.kind in 'biufc' else x)

In [8]:
data_mean = fill_mean(data)

In [9]:
data_mean.isna().sum()

Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                  0
Drivetrain            111
Length                  0
Width                   0
Height                  0
Seating Capacity        0
Fuel Tank Capacity      0
Max Power BHP           0
Max Power RPM           0
Max Torque Nm           0
Max Torque RPM          0
dtype: int64

--> Các cột với kiểu dữ liệu str không thể được fill

In [10]:
data_mean = drop_na(data_mean)
data_mean.to_csv(path + 'train_mean.csv', index=False)

### 2.2 Fill missing values with median

In [11]:
def fill_median(df : pd.DataFrame):
    return df.apply(lambda x: x.fillna(x.median()) if x.dtype.kind in 'biufc' else x)

In [12]:
data_median = fill_median(data)

In [13]:
data_median.isna().sum()

Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                  0
Drivetrain            111
Length                  0
Width                   0
Height                  0
Seating Capacity        0
Fuel Tank Capacity      0
Max Power BHP           0
Max Power RPM           0
Max Torque Nm           0
Max Torque RPM          0
dtype: int64

--> Tương tự với mean, các cột có kiểu dữ liệu str không thể được fill

In [14]:
data_median = drop_na(data_median)
data_median.to_csv(path + 'train_median.csv', index=False)

### 2.3 Fill missing values with most common value

In [15]:
def fill_mode(df : pd.DataFrame):
    return df.apply(lambda x: x.fillna(x.mode()[0]))

In [16]:
data_mode = fill_mode(data)

In [17]:
data_mode.isna().sum()

Make                  0
Model                 0
Price                 0
Year                  0
Kilometer             0
Fuel Type             0
Transmission          0
Location              0
Color                 0
Owner                 0
Seller Type           0
Engine                0
Drivetrain            0
Length                0
Width                 0
Height                0
Seating Capacity      0
Fuel Tank Capacity    0
Max Power BHP         0
Max Power RPM         0
Max Torque Nm         0
Max Torque RPM        0
dtype: int64

Không còn missing value

In [18]:
data_mode = drop_na(data_mode)
data_mode.to_csv(path + 'train_mode.csv', index=False)

### 2.4 K - Nearest Neighbour

In [19]:
def euclidean_distance(row1, row2):
    """
    Computes the Euclidean distance between two rows, ignoring NaN values and handling only numeric data.
    Non-numeric data is excluded from the distance calculation.
    """
    numeric_mask = pd.api.types.is_numeric_dtype(row1) and pd.api.types.is_numeric_dtype(row2)

    if numeric_mask:
        mask = ~np.isnan(row1) & ~np.isnan(row2)
        if np.any(mask):
            return np.sqrt(np.sum((row1[mask] - row2[mask]) ** 2))
        else:
            return np.inf
    else:
        return np.inf

def fill_knn(df, k=3):
    df_filled = df.copy()

    for i, row in df_filled.iterrows():
        if row.isnull().any():
            distances = []

            for j, other_row in df_filled.iterrows():
                if not other_row.isnull().any():
                    dist = euclidean_distance(row.values, other_row.values)
                    distances.append((dist, j))

            distances = sorted(distances, key=lambda x: x[0])
            nearest_neighbors = [df_filled.iloc[j] for dist, j in distances[:k]]

            for col in df_filled.columns:
                if pd.isnull(df_filled.at[i, col]):
                    neighbor_values = [neighbor[col] for neighbor in nearest_neighbors]
                    neighbor_values = [val for val in neighbor_values if not pd.isnull(val)]

                    if pd.api.types.is_numeric_dtype(df_filled[col]):
                        df_filled.at[i, col] = np.mean(neighbor_values)
                    else:
                        df_filled.at[i, col] = Counter(neighbor_values).most_common(1)[0][0]

    return df_filled

In [20]:
data_knn = fill_knn(data)

In [21]:
data_knn.isna().sum()

Make                  0
Model                 0
Price                 0
Year                  0
Kilometer             0
Fuel Type             0
Transmission          0
Location              0
Color                 0
Owner                 0
Seller Type           0
Engine                0
Drivetrain            0
Length                0
Width                 0
Height                0
Seating Capacity      0
Fuel Tank Capacity    0
Max Power BHP         0
Max Power RPM         0
Max Torque Nm         0
Max Torque RPM        0
dtype: int64

In [22]:
data_knn = drop_na(data_knn)
data_knn.to_csv(path + 'train_knn.csv', index=False)

### 2.5 Interpolation

In [23]:
inferred_data = data.infer_objects(copy=False)

data_interpolate_lin = inferred_data.interpolate(method='linear', limit_direction='both')
data_interpolate_lin.isna().sum()

Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                  0
Drivetrain            111
Length                  0
Width                   0
Height                  0
Seating Capacity        0
Fuel Tank Capacity      0
Max Power BHP           0
Max Power RPM           0
Max Torque Nm           0
Max Torque RPM          0
dtype: int64

In [24]:
inferred_data = data.infer_objects(copy=False)

data_interpolate_pol = inferred_data.interpolate(method='polynomial', order=2)
data_interpolate_pol.isna().sum()

Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                  1
Drivetrain            111
Length                  1
Width                   1
Height                  1
Seating Capacity        1
Fuel Tank Capacity      1
Max Power BHP           1
Max Power RPM           1
Max Torque Nm           1
Max Torque RPM          1
dtype: int64

In [25]:
inferred_data = data.infer_objects(copy=False)

data_interpolate_pad = inferred_data.interpolate(method='pad', limit=2)
data_interpolate_pad.isna().sum()

Make                  0
Model                 0
Price                 0
Year                  0
Kilometer             0
Fuel Type             0
Transmission          0
Location              0
Color                 0
Owner                 0
Seller Type           0
Engine                1
Drivetrain            1
Length                1
Width                 1
Height                1
Seating Capacity      1
Fuel Tank Capacity    1
Max Power BHP         1
Max Power RPM         1
Max Torque Nm         1
Max Torque RPM        1
dtype: int64

In [26]:
data_interpolate_lin = drop_na(data_interpolate_lin)
data_interpolate_lin.to_csv(path + 'train_interpolate_lin.csv', index=False)

data_interpolate_pol = drop_na(data_interpolate_pol)
data_interpolate_pol.to_csv(path + 'train_interpolate_pol.csv', index=False)

data_interpolate_pad = drop_na(data_interpolate_pad)
data_interpolate_pad.to_csv(path + 'train_interpolate_pad.csv', index=False)